In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.utils.data_utils import get_file
from keras.layers import Input, Dense
import time
import numpy as np
from keras import backend as K
import audio_processor as ap
import pdb


##
def sort_result(tags, preds):
    result = zip(tags, preds)
    sorted_result = sorted(result, key=lambda x: x[1], reverse=True)
    return [(name, '%5.3f' % score) for name, score in sorted_result]

def librosa_exists():
    try:
        __import__('librosa')
    except ImportError:
        return False
    else:
        return True

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import librosa

In [3]:
    audio_paths = ['data/bensound-cute.mp3',
                   'data/bensound-actionable.mp3',
                   'data/bensound-dubstep.mp3',
                   'data/bensound-thejazzpiano.mp3',
                  'data/Ballata1.mp3',
                  'data/chp_op31.mp3']
    melgram_paths = ['data/bensound-cute.npy',
                     'data/bensound-actionable.npy',
                     'data/bensound-dubstep.npy',
                     'data/bensound-thejazzpiano.npy',
                    'data/Ballata1.npy']

    tags = ['rock', 'pop', 'alternative', 'indie', 'electronic',
            'female vocalists', 'dance', '00s', 'alternative rock', 'jazz',
            'beautiful', 'metal', 'chillout', 'male vocalists',
            'classic rock', 'soul', 'indie rock', 'Mellow', 'electronica',
            '80s', 'folk', '90s', 'chill', 'instrumental', 'punk',
            'oldies', 'blues', 'hard rock', 'ambient', 'acoustic',
            'experimental', 'female vocalist', 'guitar', 'Hip-Hop',
            '70s', 'party', 'country', 'easy listening',
            'sexy', 'catchy', 'funk', 'electro', 'heavy metal',
            'Progressive rock', '60s', 'rnb', 'indie pop',
            'sad', 'House', 'happy']

    # prepare data like this
    melgrams = np.zeros((0, 1, 96, 1366))

    if librosa_exists:
        for audio_path in audio_paths:
            melgram = ap.compute_melgram(audio_path)
            melgrams = np.concatenate((melgrams, melgram), axis=0)
    else:
        for melgram_path in melgram_paths:
            melgram = np.load(melgram_path)
            melgrams = np.concatenate((melgrams, melgram), axis=0)
            
    melgrams = melgrams.transpose(0,2,3,1)
    TH_WEIGHTS_PATH = 'https://github.com/keunwoochoi/music-auto_tagging-keras/blob/master/data/music_tagger_cnn_weights_theano.h5'
    weights='msd'
    input_tensor=None
    include_top=True
    
    if weights not in {'msd', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `msd` '
                         '(pre-training on Million Song Dataset).')
    if K.image_dim_ordering() == 'th':
        input_shape = (1, 96, 1366)
    else:
        input_shape = (96, 1366, 1)

    if input_tensor is None:
        melgram_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            melgram_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            melgram_input = input_tensor
            
    if K.image_dim_ordering() == 'th':
        channel_axis = 1
        freq_axis = 2
        time_axis = 3
    else:
        channel_axis = 3
        freq_axis = 1
        time_axis = 2
        
    x = BatchNormalization(axis=freq_axis, name='bn_0_freq')(melgram_input)
    
    x = Convolution2D(32, 3, 3, border_mode='same', name='conv1')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn1')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool1')(x)
    
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv2')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn2')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool2')(x)
    
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv3')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn3')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool3')(x)
    
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv4')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn4')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(3, 5), name='pool4')(x)
    
    x = Convolution2D(32, 3, 3, border_mode='same', name='conv5')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn5')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 4), name='pool5')(x)
    
    x = Flatten()(x)
    if include_top:
        x = Dense(50, activation='sigmoid', name='output')(x)
    model = Model(melgram_input, x)
    print (model)
    #model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.save('transfer_model.h5')
   
    
   # if weights is None:
   #   return model    
   # else: 
        # Load input
      #  if K.image_dim_ordering() == 'tf':
         #   raise RuntimeError("Please set image_dim_ordering == 'th'."
            #                   "You can set it at ~/.keras/keras.json")
           # model.load_weights('data/music_tagger_cnn_weights_%s.h5' % K._BACKEND,
                 #          by_name=True)
            

            # predict the tags like this
    print('Predicting...')
    start = time.time()
    pred_tags = model.predict(melgrams)
    # print like this...
  #  print "Prediction is done. It took %d seconds." % (time.time()-start)
    print('Printing top-10 tags for each track...')
    
    
    for song_idx, audio_path in enumerate(audio_paths):
        sorted_result = sort_result(tags, pred_tags[song_idx, :].tolist())
        print(audio_path)
        print(sorted_result[:5])
        print(sorted_result[5:10])
        print(' ')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), name="conv1", padding="same")`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=3, name="bn1")`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="conv2", padding="same")`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=3, name="bn2")`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="conv3", padding="same")`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_

Predicting...
Printing top-10 tags for each track...
data/bensound-cute.mp3
[('electronic', '1.000'), ('folk', '0.999'), ('electronica', '0.999'), ('chill', '0.997'), ('60s', '0.997')]
[('beautiful', '0.997'), ('sad', '0.984'), ('indie rock', '0.966'), ('70s', '0.956'), ('alternative', '0.956')]
 
data/bensound-actionable.mp3
[('electronic', '0.999'), ('60s', '0.997'), ('chill', '0.997'), ('electronica', '0.997'), ('folk', '0.991')]
[('beautiful', '0.989'), ('sad', '0.969'), ('indie rock', '0.968'), ('70s', '0.960'), ('country', '0.957')]
 
data/bensound-dubstep.mp3
[('electronic', '1.000'), ('electronica', '1.000'), ('folk', '0.999'), ('60s', '0.999'), ('chill', '0.999')]
[('beautiful', '0.995'), ('sad', '0.994'), ('indie rock', '0.989'), ('alternative', '0.984'), ('country', '0.971')]
 
data/bensound-thejazzpiano.mp3
[('electronic', '0.999'), ('chill', '0.997'), ('folk', '0.997'), ('60s', '0.995'), ('electronica', '0.995')]
[('beautiful', '0.994'), ('70s', '0.968'), ('sad', '0.967'),

# Test Part I

In [8]:
import glob

In [220]:
len(glob.glob("data/mini_test/*.mp3"))

20

In [221]:
melgrams_t = np.zeros((0, 1, 96, 1366))
for i in sorted(glob.glob("data/mini_test/*.mp3")):
    melgram_t = ap.compute_melgram(i)
    melgrams_t = np.concatenate((melgrams_t, melgram_t), axis=0)

In [222]:
melgrams_t = melgrams_t.transpose(0,2,3,1)

In [223]:
    pred_test = model.predict(melgrams_t)

In [225]:
    chopin_result = []
    other_result = []
    for song_idx, audio_path in enumerate(sorted(glob.glob("data/mini_test/*.mp3"))):
        sorted_result = sort_result(tags, pred_test[song_idx, :].tolist())
        if song_idx < 11: 
            chopin_result.append(sorted_result)
        else:
            other_result.append(sorted_result)
        print(audio_path)
        print(sorted_result[:5])
        print(sorted_result[5:10])
        print(' ')

data/mini_test/Chopin - Andante spinato.mp3
[('chillout', '0.994'), ('oldies', '0.992'), ('rock', '0.989'), ('electronica', '0.987'), ('catchy', '0.981')]
[('alternative rock', '0.969'), ('jazz', '0.968'), ('70s', '0.955'), ('heavy metal', '0.945'), ('experimental', '0.940')]
 
data/mini_test/Chopin - Ballade in A flat major op.47 no.3.mp3
[('oldies', '1.000'), ('chillout', '0.999'), ('electronica', '0.999'), ('rock', '0.999'), ('catchy', '0.997')]
[('jazz', '0.996'), ('alternative rock', '0.995'), ('70s', '0.985'), ('heavy metal', '0.985'), ('experimental', '0.984')]
 
data/mini_test/Chopin - Ballade in F major op.38 No. 2.mp3
[('oldies', '0.999'), ('chillout', '0.999'), ('rock', '0.998'), ('electronica', '0.998'), ('catchy', '0.996')]
[('jazz', '0.996'), ('alternative rock', '0.993'), ('70s', '0.986'), ('experimental', '0.986'), ('heavy metal', '0.980')]
 
data/mini_test/Chopin - Ballade in G minor, op.23 no.1.mp3
[('oldies', '0.995'), ('electronica', '0.995'), ('chillout', '0.994'),

In [7]:
import pandas as pd

In [227]:
chopin_result

[[('chillout', '0.994'),
  ('oldies', '0.992'),
  ('rock', '0.989'),
  ('electronica', '0.987'),
  ('catchy', '0.981'),
  ('alternative rock', '0.969'),
  ('jazz', '0.968'),
  ('70s', '0.955'),
  ('heavy metal', '0.945'),
  ('experimental', '0.940'),
  ('ambient', '0.851'),
  ('dance', '0.839'),
  ('alternative', '0.806'),
  ('indie', '0.784'),
  ('instrumental', '0.773'),
  ('acoustic', '0.754'),
  ('electronic', '0.727'),
  ('party', '0.707'),
  ('00s', '0.653'),
  ('indie rock', '0.646'),
  ('80s', '0.617'),
  ('Hip-Hop', '0.603'),
  ('blues', '0.579'),
  ('Progressive rock', '0.524'),
  ('guitar', '0.507'),
  ('folk', '0.493'),
  ('punk', '0.470'),
  ('beautiful', '0.458'),
  ('male vocalists', '0.445'),
  ('90s', '0.404'),
  ('soul', '0.393'),
  ('country', '0.347'),
  ('sexy', '0.343'),
  ('60s', '0.317'),
  ('pop', '0.283'),
  ('Mellow', '0.259'),
  ('rnb', '0.240'),
  ('happy', '0.139'),
  ('sad', '0.097'),
  ('metal', '0.069'),
  ('female vocalists', '0.053'),
  ('female vocal

In [228]:
new_chopin = []
for i in range(len(chopin_result)):
    new_chopin.append(sorted(chopin_result[i]))

In [229]:
new_other = []
for i in range(len(other_result)):
    new_other.append(sorted(other_result[i]))

In [230]:
mus_index = []
for i in range(len(new_chopin[0])):
    mus_index.append(new_chopin[0][i][0])
    

In [231]:
len(new_chopin)

11

In [232]:
mus_rank = []
for i in range(len(new_chopin[0])):
    mus_rank.append(new_chopin[0][i][0])

In [233]:
new_df = pd.DataFrame(new_chopin, columns=mus_index)

In [235]:
    for tag in new_df.keys():
        for num in range(new_df.shape[0]):
            new_df[tag][num] = new_df[tag][num][1]

In [236]:
new_df["Composer"] = ["Chopin"]*11

In [237]:
new_df

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,oldies,party,pop,punk,rnb,rock,sad,sexy,soul,Composer
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393,Chopin
1,0.534,0.188,0.985,0.818,0.254,0.534,0.000,0.088,0.356,0.907,...,1.000,0.854,0.249,0.442,0.092,0.999,0.030,0.323,0.234,Chopin
2,0.548,0.212,0.986,0.769,0.322,0.509,0.000,0.114,0.353,0.892,...,0.999,0.865,0.254,0.482,0.103,0.998,0.034,0.342,0.218,Chopin
3,0.653,0.182,0.966,0.720,0.303,0.647,0.004,0.124,0.339,0.751,...,0.995,0.734,0.314,0.571,0.143,0.986,0.075,0.303,0.471,Chopin
4,0.565,0.255,0.967,0.740,0.316,0.614,0.002,0.179,0.394,0.787,...,0.997,0.763,0.315,0.475,0.173,0.993,0.065,0.344,0.361,Chopin
5,0.696,0.434,0.904,0.464,0.464,0.469,0.038,0.435,0.507,0.806,...,0.924,0.697,0.415,0.617,0.331,0.971,0.087,0.466,0.369,Chopin
6,0.473,0.237,0.979,0.715,0.216,0.678,0.027,0.217,0.330,0.739,...,0.978,0.804,0.374,0.429,0.215,0.944,0.174,0.659,0.479,Chopin
7,0.676,0.315,0.957,0.605,0.380,0.565,0.017,0.299,0.397,0.784,...,0.970,0.715,0.390,0.643,0.274,0.974,0.096,0.436,0.344,Chopin
8,0.622,0.364,0.962,0.487,0.415,0.686,0.022,0.365,0.510,0.772,...,0.955,0.686,0.375,0.493,0.288,0.984,0.110,0.527,0.557,Chopin
9,0.667,0.232,0.980,0.757,0.362,0.698,0.011,0.296,0.297,0.781,...,0.981,0.736,0.344,0.650,0.233,0.980,0.041,0.475,0.270,Chopin


In [238]:
other_df=pd.DataFrame(new_other, columns=mus_index)
for tag in other_df.keys():
    for num in range(other_df.shape[0]):
        other_df[tag][num] = other_df[tag][num][1]
other_df

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.453,0.185,0.967,0.795,0.340,0.438,0.023,0.113,0.078,0.698,...,0.077,0.986,0.808,0.572,0.493,0.167,0.846,0.146,0.580,0.625
1,0.523,0.217,0.975,0.698,0.399,0.523,0.032,0.296,0.323,0.808,...,0.076,0.967,0.836,0.446,0.564,0.180,0.973,0.069,0.536,0.268
2,0.355,0.492,0.872,0.705,0.130,0.534,0.069,0.126,0.134,0.876,...,0.188,0.960,0.749,0.501,0.482,0.236,0.936,0.071,0.562,0.467
3,0.474,0.270,0.963,0.723,0.476,0.572,0.038,0.249,0.199,0.678,...,0.052,0.957,0.767,0.550,0.531,0.255,0.869,0.166,0.643,0.547
4,0.730,0.263,0.946,0.581,0.481,0.474,0.010,0.255,0.470,0.693,...,0.081,0.986,0.737,0.347,0.620,0.280,0.982,0.139,0.347,0.360
5,0.340,0.195,0.949,0.741,0.357,0.585,0.024,0.156,0.210,0.844,...,0.084,0.988,0.838,0.560,0.316,0.115,0.936,0.191,0.639,0.679
6,0.588,0.178,0.977,0.742,0.385,0.604,0.021,0.309,0.352,0.795,...,0.064,0.975,0.795,0.377,0.543,0.155,0.977,0.072,0.511,0.296
7,0.324,0.326,0.958,0.671,0.207,0.546,0.029,0.146,0.219,0.861,...,0.080,0.968,0.807,0.472,0.524,0.180,0.959,0.054,0.478,0.353
8,0.174,0.367,0.880,0.745,0.136,0.577,0.046,0.086,0.035,0.853,...,0.280,0.990,0.873,0.640,0.162,0.130,0.955,0.104,0.630,0.654


In [239]:
other_df["Composer"] = ["Other"]*other_df.shape[0]

In [241]:
mix_df = new_df.append(other_df)

In [243]:
composer_raw = mix_df['Composer']
features_raw = mix_df.drop('Composer', axis = 1)

In [244]:
mix_df.head()

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,oldies,party,pop,punk,rnb,rock,sad,sexy,soul,Composer
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393,Chopin
1,0.534,0.188,0.985,0.818,0.254,0.534,0.000,0.088,0.356,0.907,...,1.000,0.854,0.249,0.442,0.092,0.999,0.030,0.323,0.234,Chopin
2,0.548,0.212,0.986,0.769,0.322,0.509,0.000,0.114,0.353,0.892,...,0.999,0.865,0.254,0.482,0.103,0.998,0.034,0.342,0.218,Chopin
3,0.653,0.182,0.966,0.720,0.303,0.647,0.004,0.124,0.339,0.751,...,0.995,0.734,0.314,0.571,0.143,0.986,0.075,0.303,0.471,Chopin
4,0.565,0.255,0.967,0.740,0.316,0.614,0.002,0.179,0.394,0.787,...,0.997,0.763,0.315,0.475,0.173,0.993,0.065,0.344,0.361,Chopin


In [245]:
mix_df.drop('Composer', axis = 1).head()

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.069,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393
1,0.534,0.188,0.985,0.818,0.254,0.534,0.000,0.088,0.356,0.907,...,0.021,1.000,0.854,0.249,0.442,0.092,0.999,0.030,0.323,0.234
2,0.548,0.212,0.986,0.769,0.322,0.509,0.000,0.114,0.353,0.892,...,0.021,0.999,0.865,0.254,0.482,0.103,0.998,0.034,0.342,0.218
3,0.653,0.182,0.966,0.720,0.303,0.647,0.004,0.124,0.339,0.751,...,0.049,0.995,0.734,0.314,0.571,0.143,0.986,0.075,0.303,0.471
4,0.565,0.255,0.967,0.740,0.316,0.614,0.002,0.179,0.394,0.787,...,0.049,0.997,0.763,0.315,0.475,0.173,0.993,0.065,0.344,0.361


In [247]:
features_final = features_raw

In [248]:
features_final

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.069,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393
1,0.534,0.188,0.985,0.818,0.254,0.534,0.000,0.088,0.356,0.907,...,0.021,1.000,0.854,0.249,0.442,0.092,0.999,0.030,0.323,0.234
2,0.548,0.212,0.986,0.769,0.322,0.509,0.000,0.114,0.353,0.892,...,0.021,0.999,0.865,0.254,0.482,0.103,0.998,0.034,0.342,0.218
3,0.653,0.182,0.966,0.720,0.303,0.647,0.004,0.124,0.339,0.751,...,0.049,0.995,0.734,0.314,0.571,0.143,0.986,0.075,0.303,0.471
4,0.565,0.255,0.967,0.740,0.316,0.614,0.002,0.179,0.394,0.787,...,0.049,0.997,0.763,0.315,0.475,0.173,0.993,0.065,0.344,0.361
5,0.696,0.434,0.904,0.464,0.464,0.469,0.038,0.435,0.507,0.806,...,0.131,0.924,0.697,0.415,0.617,0.331,0.971,0.087,0.466,0.369
6,0.473,0.237,0.979,0.715,0.216,0.678,0.027,0.217,0.330,0.739,...,0.054,0.978,0.804,0.374,0.429,0.215,0.944,0.174,0.659,0.479
7,0.676,0.315,0.957,0.605,0.380,0.565,0.017,0.299,0.397,0.784,...,0.062,0.970,0.715,0.390,0.643,0.274,0.974,0.096,0.436,0.344
8,0.622,0.364,0.962,0.487,0.415,0.686,0.022,0.365,0.510,0.772,...,0.092,0.955,0.686,0.375,0.493,0.288,0.984,0.110,0.527,0.557
9,0.667,0.232,0.980,0.757,0.362,0.698,0.011,0.296,0.297,0.781,...,0.050,0.981,0.736,0.344,0.650,0.233,0.980,0.041,0.475,0.270


In [249]:
composer = np.where(composer_raw.str.contains("Other"),0,1)

In [250]:
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

50 total features after one-hot encoding.


In [251]:
print (encoded)

['00s', '60s', '70s', '80s', '90s', 'Hip-Hop', 'House', 'Mellow', 'Progressive rock', 'acoustic', 'alternative', 'alternative rock', 'ambient', 'beautiful', 'blues', 'catchy', 'chill', 'chillout', 'classic rock', 'country', 'dance', 'easy listening', 'electro', 'electronic', 'electronica', 'experimental', 'female vocalist', 'female vocalists', 'folk', 'funk', 'guitar', 'happy', 'hard rock', 'heavy metal', 'indie', 'indie pop', 'indie rock', 'instrumental', 'jazz', 'male vocalists', 'metal', 'oldies', 'party', 'pop', 'punk', 'rnb', 'rock', 'sad', 'sexy', 'soul']


In [252]:

# Import train_test_split# Import 
from sklearn.cross_validation import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    composer, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 16 samples.
Testing set has 4 samples.


In [253]:
TP = np.sum(composer) # Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data 
#encoded to numerical values done in the data preprocessing step.
FP = len(composer) - TP # Specific to the naive case

TN = 0 # No predicted negatives in the naive case
FN = 0 # No predicted negatives in the naive case

# TODO: Calculate accuracy, precision and recall
accuracy = TP / (TP+FP)
recall = 1
precision = accuracy

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
fscore = ((1+0.5**2))*precision*recall/((0.5**2*precision)+recall)

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

Naive Predictor: [Accuracy score: 0.5500, F-score: 0.6044]


In [33]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score,accuracy_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size],  y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end-start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, beta=0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [6]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

# Import supplementary visualization code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

In [256]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier




# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
# HINT: samples_1 is 1% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
samples_100 = len(y_train)
#samples_10 = int(0.1*samples_100)
#samples_1 = int(0.01*samples_100)

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
#vs.evaluate(results, accuracy, fscore)

GaussianNB trained on 16 samples.
DecisionTreeClassifier trained on 16 samples.
AdaBoostClassifier trained on 16 samples.


In [257]:
results

{'GaussianNB': {0: {'train_time': 0.0013921260833740234,
   'pred_time': 0.0007500648498535156,
   'acc_train': 0.875,
   'acc_test': 1.0,
   'f_train': 0.8490566037735848,
   'f_test': 1.0}},
 'DecisionTreeClassifier': {0: {'train_time': 0.0009059906005859375,
   'pred_time': 0.00044655799865722656,
   'acc_train': 1.0,
   'acc_test': 1.0,
   'f_train': 1.0,
   'f_test': 1.0}},
 'AdaBoostClassifier': {0: {'train_time': 0.06002640724182129,
   'pred_time': 0.00851130485534668,
   'acc_train': 1.0,
   'acc_test': 1.0,
   'f_train': 1.0,
   'f_test': 1.0}}}

In [263]:
features_raw.loc[0]

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.069,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393
0,0.453,0.185,0.967,0.795,0.340,0.438,0.023,0.113,0.078,0.698,...,0.077,0.986,0.808,0.572,0.493,0.167,0.846,0.146,0.580,0.625


In [266]:
features_raw.loc[0]

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.653,0.317,0.955,0.617,0.404,0.603,0.006,0.259,0.524,0.754,...,0.069,0.992,0.707,0.283,0.470,0.240,0.989,0.097,0.343,0.393
0,0.453,0.185,0.967,0.795,0.340,0.438,0.023,0.113,0.078,0.698,...,0.077,0.986,0.808,0.572,0.493,0.167,0.846,0.146,0.580,0.625


In [282]:
other_df

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,oldies,party,pop,punk,rnb,rock,sad,sexy,soul,Composer
0,0.453,0.185,0.967,0.795,0.340,0.438,0.023,0.113,0.078,0.698,...,0.986,0.808,0.572,0.493,0.167,0.846,0.146,0.580,0.625,Other
1,0.523,0.217,0.975,0.698,0.399,0.523,0.032,0.296,0.323,0.808,...,0.967,0.836,0.446,0.564,0.180,0.973,0.069,0.536,0.268,Other
2,0.355,0.492,0.872,0.705,0.130,0.534,0.069,0.126,0.134,0.876,...,0.960,0.749,0.501,0.482,0.236,0.936,0.071,0.562,0.467,Other
3,0.474,0.270,0.963,0.723,0.476,0.572,0.038,0.249,0.199,0.678,...,0.957,0.767,0.550,0.531,0.255,0.869,0.166,0.643,0.547,Other
4,0.730,0.263,0.946,0.581,0.481,0.474,0.010,0.255,0.470,0.693,...,0.986,0.737,0.347,0.620,0.280,0.982,0.139,0.347,0.360,Other
5,0.340,0.195,0.949,0.741,0.357,0.585,0.024,0.156,0.210,0.844,...,0.988,0.838,0.560,0.316,0.115,0.936,0.191,0.639,0.679,Other
6,0.588,0.178,0.977,0.742,0.385,0.604,0.021,0.309,0.352,0.795,...,0.975,0.795,0.377,0.543,0.155,0.977,0.072,0.511,0.296,Other
7,0.324,0.326,0.958,0.671,0.207,0.546,0.029,0.146,0.219,0.861,...,0.968,0.807,0.472,0.524,0.180,0.959,0.054,0.478,0.353,Other
8,0.174,0.367,0.880,0.745,0.136,0.577,0.046,0.086,0.035,0.853,...,0.990,0.873,0.640,0.162,0.130,0.955,0.104,0.630,0.654,Other


In [303]:
other_df.loc[0:1].drop('Composer', axis = 1).shape

(2, 50)

In [305]:
papa = (other_df.loc[0].drop('Composer')).reshape(1,50)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [306]:
papa.shape

(1, 50)

In [313]:
papa

array([['0.453', '0.185', '0.967', '0.795', '0.340', '0.438', '0.023',
        '0.113', '0.078', '0.698', '0.689', '0.987', '0.761', '0.243',
        '0.635', '0.807', '0.080', '0.977', '0.122', '0.397', '0.811',
        '0.166', '0.032', '0.643', '0.983', '0.675', '0.056', '0.040',
        '0.902', '0.208', '0.213', '0.089', '0.145', '0.956', '0.709',
        '0.062', '0.251', '0.784', '0.908', '0.180', '0.077', '0.986',
        '0.808', '0.572', '0.493', '0.167', '0.846', '0.146', '0.580',
        '0.625']], dtype=object)

In [311]:
np.array(new_chopin[0]).shape

(50, 2)

In [307]:
clf_A.predict(papa)

array([0])

In [260]:
results

{'GaussianNB': {0: {'train_time': 0.0013921260833740234,
   'pred_time': 0.0007500648498535156,
   'acc_train': 0.875,
   'acc_test': 1.0,
   'f_train': 0.8490566037735848,
   'f_test': 1.0}},
 'DecisionTreeClassifier': {0: {'train_time': 0.0009059906005859375,
   'pred_time': 0.00044655799865722656,
   'acc_train': 1.0,
   'acc_test': 1.0,
   'f_train': 1.0,
   'f_test': 1.0}},
 'AdaBoostClassifier': {0: {'train_time': 0.06002640724182129,
   'pred_time': 0.00851130485534668,
   'acc_train': 1.0,
   'acc_test': 1.0,
   'f_train': 1.0,
   'f_test': 1.0}}}

In [262]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# TODO: Initialize the classifier
clf = AdaBoostClassifier(random_state=0)

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# HINT: parameters = {'parameter_1': [value1, value2], 'parameter_2': [value1, value2]}
parameters = {'n_estimators':[100,150,200,50],'learning_rate':[0.1,0.3,0.5,0.8,1]}

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta=0.5)

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(clf, parameters,scoring=scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

Unoptimized model
------
Accuracy score on testing data: 1.0000
F-score on testing data: 1.0000

Optimized Model
------
Final accuracy score on the testing data: 1.0000
Final F-score on the testing data: 1.0000


# Final Part

In [9]:
melgrams_t = np.zeros((0, 1, 96, 1366))
for i in sorted(glob.glob("chopin_for_classification/*.mp3")):
    melgram_t = ap.compute_melgram(i)
    melgrams_t = np.concatenate((melgrams_t, melgram_t), axis=0)
melgrams_t = melgrams_t.transpose(0,2,3,1)

In [10]:
    pred_test_new = model.predict(melgrams_t)

In [11]:
    chopin_updated_result = []
    for song_idx, audio_path in enumerate(sorted(glob.glob("chopin_for_classification/*.mp3"))):
        sorted_result_new = sort_result(tags, pred_test_new[song_idx, :].tolist())
        chopin_updated_result.append(sorted_result_new)
        print(audio_path)
        print(sorted_result_new[:5])
        print(sorted_result_new[5:10])
        print(' ')

chopin_for_classification/Allegro de Concert Op. 46 in A Major.mp3
[('electronic', '0.999'), ('electronica', '0.998'), ('60s', '0.996'), ('70s', '0.977'), ('chill', '0.975')]
[('funk', '0.970'), ('folk', '0.969'), ('sexy', '0.962'), ('beautiful', '0.962'), ('alternative rock', '0.958')]
 
chopin_for_classification/Ballade no. 1 - Op. 23.mp3
[('electronic', '1.000'), ('electronica', '0.997'), ('60s', '0.997'), ('chill', '0.996'), ('beautiful', '0.993')]
[('70s', '0.983'), ('folk', '0.982'), ('sad', '0.945'), ('funk', '0.942'), ('indie rock', '0.912')]
 
chopin_for_classification/Ballade no. 2 - Op. 38.mp3
[('electronica', '0.995'), ('electronic', '0.995'), ('60s', '0.987'), ('folk', '0.956'), ('alternative rock', '0.947')]
[('chill', '0.933'), ('funk', '0.930'), ('70s', '0.909'), ('male vocalists', '0.899'), ('blues', '0.891')]
 
chopin_for_classification/Ballade no. 3 - Op. 47.mp3
[('electronic', '0.999'), ('electronica', '0.999'), ('60s', '0.997'), ('folk', '0.987'), ('chill', '0.976'

In [12]:
len(chopin_updated_result)

104

In [13]:
len(glob.glob("other/other/*.mp3")[:-27])

104

In [14]:
melgrams_other = np.zeros((0, 1, 96, 1366))
for i in sorted(glob.glob("other/other/*.mp3")[:-27]):
    melgram_other = ap.compute_melgram(i)
    melgrams_other = np.concatenate((melgrams_other, melgram_other), axis=0)
melgrams_other = melgrams_other.transpose(0,2,3,1)

In [71]:
melgrams_other.shape

(104, 96, 1366, 1)

In [15]:
    pred_test_new_other = model.predict(melgrams_other)

In [17]:
    other_updated_result = []
    for song_idx, audio_path in enumerate(sorted(glob.glob("other/other/*.mp3")[:-27])):
        sorted_result_new = sort_result(tags, pred_test_new_other[song_idx, :].tolist())
        other_updated_result.append(sorted_result_new)
        print(audio_path)
        print(sorted_result_new[:5])
        print(sorted_result_new[5:10])
        print(' ')

other/other/other (100).mp3
[('electronic', '0.998'), ('electronica', '0.995'), ('60s', '0.988'), ('chill', '0.984'), ('70s', '0.957')]
[('folk', '0.954'), ('beautiful', '0.947'), ('funk', '0.943'), ('sad', '0.916'), ('jazz', '0.891')]
 
other/other/other (101).mp3
[('electronica', '0.997'), ('electronic', '0.997'), ('60s', '0.992'), ('folk', '0.970'), ('chill', '0.962')]
[('blues', '0.936'), ('funk', '0.927'), ('70s', '0.922'), ('male vocalists', '0.919'), ('alternative rock', '0.905')]
 
other/other/other (103).mp3
[('electronic', '0.999'), ('electronica', '0.995'), ('chill', '0.994'), ('60s', '0.990'), ('beautiful', '0.986')]
[('folk', '0.975'), ('sad', '0.952'), ('70s', '0.937'), ('indie rock', '0.923'), ('funk', '0.887')]
 
other/other/other (104).mp3
[('electronic', '0.997'), ('electronica', '0.994'), ('60s', '0.992'), ('folk', '0.972'), ('chill', '0.955')]
[('70s', '0.925'), ('funk', '0.924'), ('sad', '0.917'), ('blues', '0.917'), ('beautiful', '0.907')]
 
other/other/other (105

In [18]:
len(other_updated_result)

104

In [21]:
ultimate_chopin = []
for i in range(len(chopin_updated_result)):
    ultimate_chopin.append(sorted(chopin_updated_result[i]))
ultimate_other = []
for i in range(len(other_updated_result)):
    ultimate_other.append(sorted(other_updated_result[i]))
new_mus_index = []
for i in range(len(ultimate_chopin[0])):
    new_mus_index.append(ultimate_chopin[0][i][0])
new_mus_rank = []
for i in range(len(ultimate_chopin[0])):
    new_mus_rank.append(ultimate_chopin[0][i][0])
ultimate_chopin_df = pd.DataFrame(ultimate_chopin, columns=new_mus_index)
for tag in ultimate_chopin_df.keys():
    for num in range(ultimate_chopin_df.shape[0]):
        ultimate_chopin_df[tag][num] = ultimate_chopin_df[tag][num][1]
ultimate_chopin_df["Composer"] = ["Chopin"]*ultimate_chopin_df.shape[0]

In [23]:
ultimate_other_df=pd.DataFrame(ultimate_other, columns=new_mus_index)
for tag in ultimate_other_df.keys():
    for num in range(ultimate_other_df.shape[0]):
        ultimate_other_df[tag][num] = ultimate_other_df[tag][num][1]
ultimate_other_df

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,metal,oldies,party,pop,punk,rnb,rock,sad,sexy,soul
0,0.348,0.988,0.957,0.006,0.190,0.225,0.813,0.146,0.853,0.474,...,0.342,0.273,0.516,0.096,0.827,0.444,0.025,0.916,0.865,0.501
1,0.311,0.992,0.922,0.007,0.147,0.160,0.823,0.410,0.758,0.306,...,0.378,0.219,0.688,0.073,0.584,0.251,0.022,0.823,0.881,0.390
2,0.176,0.990,0.937,0.004,0.351,0.065,0.791,0.078,0.881,0.504,...,0.364,0.197,0.574,0.097,0.840,0.516,0.011,0.952,0.796,0.674
3,0.262,0.992,0.925,0.005,0.222,0.172,0.767,0.246,0.825,0.379,...,0.385,0.335,0.569,0.116,0.609,0.347,0.020,0.917,0.806,0.365
4,0.324,0.995,0.945,0.002,0.153,0.174,0.820,0.182,0.877,0.410,...,0.440,0.300,0.551,0.080,0.765,0.407,0.014,0.928,0.865,0.385
5,0.258,0.992,0.972,0.005,0.224,0.329,0.874,0.223,0.774,0.426,...,0.460,0.333,0.562,0.054,0.663,0.281,0.010,0.832,0.918,0.482
6,0.270,0.994,0.952,0.006,0.143,0.255,0.813,0.302,0.753,0.374,...,0.296,0.247,0.585,0.036,0.658,0.286,0.011,0.852,0.914,0.420
7,0.305,0.979,0.928,0.006,0.322,0.146,0.753,0.106,0.870,0.543,...,0.369,0.282,0.504,0.092,0.756,0.362,0.015,0.925,0.771,0.767
8,0.306,0.988,0.946,0.017,0.142,0.205,0.791,0.512,0.672,0.394,...,0.393,0.325,0.706,0.053,0.580,0.342,0.032,0.760,0.897,0.312
9,0.382,0.984,0.915,0.002,0.374,0.056,0.855,0.108,0.915,0.795,...,0.150,0.343,0.675,0.083,0.617,0.674,0.044,0.951,0.551,0.699


In [24]:
ultimate_other_df["Composer"] = ["Other"]*ultimate_other_df.shape[0]

In [25]:
ultimate_other_df.head()

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,oldies,party,pop,punk,rnb,rock,sad,sexy,soul,Composer
0,0.348,0.988,0.957,0.006,0.190,0.225,0.813,0.146,0.853,0.474,...,0.273,0.516,0.096,0.827,0.444,0.025,0.916,0.865,0.501,Other
1,0.311,0.992,0.922,0.007,0.147,0.160,0.823,0.410,0.758,0.306,...,0.219,0.688,0.073,0.584,0.251,0.022,0.823,0.881,0.390,Other
2,0.176,0.990,0.937,0.004,0.351,0.065,0.791,0.078,0.881,0.504,...,0.197,0.574,0.097,0.840,0.516,0.011,0.952,0.796,0.674,Other
3,0.262,0.992,0.925,0.005,0.222,0.172,0.767,0.246,0.825,0.379,...,0.335,0.569,0.116,0.609,0.347,0.020,0.917,0.806,0.365,Other
4,0.324,0.995,0.945,0.002,0.153,0.174,0.820,0.182,0.877,0.410,...,0.300,0.551,0.080,0.765,0.407,0.014,0.928,0.865,0.385,Other


In [26]:
ultimate_mix_df = ultimate_chopin_df.append(ultimate_other_df)

In [27]:
ultimate_mix_df.head()

,00s,60s,70s,80s,90s,Hip-Hop,House,Mellow,Progressive rock,acoustic,...,oldies,party,pop,punk,rnb,rock,sad,sexy,soul,Composer
0,0.206,0.996,0.977,0.001,0.113,0.130,0.898,0.381,0.913,0.442,...,0.098,0.634,0.022,0.773,0.339,0.014,0.823,0.962,0.355,Chopin
1,0.147,0.997,0.983,0.001,0.202,0.077,0.874,0.104,0.897,0.545,...,0.176,0.602,0.028,0.901,0.397,0.010,0.945,0.881,0.501,Chopin
2,0.259,0.987,0.909,0.007,0.237,0.182,0.824,0.527,0.854,0.350,...,0.219,0.623,0.074,0.642,0.316,0.028,0.794,0.879,0.345,Chopin
3,0.213,0.997,0.968,0.002,0.176,0.108,0.908,0.589,0.827,0.356,...,0.165,0.720,0.028,0.556,0.228,0.014,0.786,0.928,0.350,Chopin
4,0.177,0.994,0.960,0.002,0.273,0.182,0.799,0.249,0.926,0.605,...,0.224,0.518,0.042,0.799,0.409,0.028,0.929,0.842,0.436,Chopin


In [28]:
ultimate_composer_raw = ultimate_mix_df['Composer']
ultimate_features_raw = ultimate_mix_df.drop('Composer', axis = 1)

In [29]:
ultimate_composer = np.where(ultimate_composer_raw.str.contains("Other"),0,1)

In [30]:

# Import train_test_split# Import 
from sklearn.cross_validation import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ultimate_features_raw, 
                                                    ultimate_composer, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 166 samples.
Testing set has 42 samples.


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [36]:
TP = np.sum(ultimate_composer) # Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data 
#encoded to numerical values done in the data preprocessing step.
FP = len(ultimate_composer) - TP # Specific to the naive case

TN = 0 # No predicted negatives in the naive case
FN = 0 # No predicted negatives in the naive case

# TODO: Calculate accuracy, precision and recall
accuracy = TP / (TP+FP)
recall = 1
precision = accuracy

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
fscore = ((1+0.5**2))*precision*recall/((0.5**2*precision)+recall)

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

Naive Predictor: [Accuracy score: 0.5000, F-score: 0.5556]


In [37]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier




# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
# HINT: samples_1 is 1% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
samples_100 = len(y_train)
samples_10 = int(0.1*samples_100)
#samples_1 = int(0.01*samples_100)

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_100,samples_10]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
#vs.evaluate(results, accuracy, fscore)

GaussianNB trained on 166 samples.
GaussianNB trained on 16 samples.
DecisionTreeClassifier trained on 166 samples.
DecisionTreeClassifier trained on 16 samples.
AdaBoostClassifier trained on 166 samples.
AdaBoostClassifier trained on 16 samples.


In [38]:
results

{'GaussianNB': {0: {'train_time': 0.006314992904663086,
   'pred_time': 0.005070924758911133,
   'acc_train': 0.6024096385542169,
   'acc_test': 0.6428571428571429,
   'f_train': 0.6010230179028133,
   'f_test': 0.6435643564356436},
  1: {'train_time': 0.0023937225341796875,
   'pred_time': 0.004304409027099609,
   'acc_train': 0.5180722891566265,
   'acc_test': 0.6428571428571429,
   'f_train': 0.4985754985754986,
   'f_test': 0.6521739130434782}},
 'DecisionTreeClassifier': {0: {'train_time': 0.009678125381469727,
   'pred_time': 0.003130197525024414,
   'acc_train': 1.0,
   'acc_test': 0.47619047619047616,
   'f_train': 1.0,
   'f_test': 0.4761904761904762},
  1: {'train_time': 0.0016412734985351562,
   'pred_time': 0.0031113624572753906,
   'acc_train': 0.5240963855421686,
   'acc_test': 0.5714285714285714,
   'f_train': 0.5410821643286573,
   'f_test': 0.5813953488372092}},
 'AdaBoostClassifier': {0: {'train_time': 0.12780451774597168,
   'pred_time': 0.010909080505371094,
   'acc

In [46]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# TODO: Initialize the classifier
clf = AdaBoostClassifier(random_state=0)

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# HINT: parameters = {'parameter_1': [value1, value2], 'parameter_2': [value1, value2]}
parameters = {'n_estimators':[100,150,200,50],'learning_rate':[0.1,0.3,0.5,0.8,1]}

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta=0.5)

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(clf, parameters,scoring=scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

Unoptimized model
------
Accuracy score on testing data: 0.4762
F-score on testing data: 0.4959

Optimized Model
------
Final accuracy score on the testing data: 0.4762
Final F-score on the testing data: 0.4959


In [47]:

# TODO: Import a supervised learning model that has 'feature_importances_'# TODO:  
from sklearn.ensemble import AdaBoostClassifier
# TODO: Train the supervised model on the training set using .fit(X_train, y_train)
model = best_clf.fit(X_train, y_train)

# TODO: Extract the feature importances using .feature_importances_ 
importances = model.feature_importances_ 

In [48]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_test_reduced = X_test[X_test.columns.values[(np.argsort(importances)[::-1])[:5]]]

# Train on the "best" model found from grid search earlier
clf = (clone(best_clf)).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

Final Model trained on full data
------
Accuracy on testing data: 0.4762
F-score on testing data: 0.4959

Final Model trained on reduced data
------
Accuracy on testing data: 0.6190
F-score on testing data: 0.6198


In [59]:
predict_test = ultimate_mix_df.loc[2].drop('Composer',axis=1)

In [133]:
test = ultimate_mix_df.loc[ultimate_mix_df['Composer']=="Chopin"].iloc[0:5]

In [134]:
X_test_reduced_2 = test[test.columns.values[(np.argsort(importances)[::-1])[:5]]]

In [135]:
X_test_reduced_2

,sad,chillout,alternative rock,folk,heavy metal
0,0.823,0.803,0.958,0.969,0.151
1,0.945,0.854,0.869,0.982,0.069
2,0.794,0.760,0.947,0.956,0.276
3,0.786,0.849,0.972,0.987,0.289
4,0.929,0.808,0.929,0.963,0.144


In [136]:
clf.predict(X_test_reduced_2)

array([1, 1, 1, 0, 1])

In [137]:
def predict(audio_path):
    melgrams_t = np.zeros((0, 1, 96, 1366))
    for i in sorted(glob.glob(audio_path)):
        melgram_t = ap.compute_melgram(i)
        melgrams_t = np.concatenate((melgrams_t, melgram_t), axis=0)
    melgrams_t = melgrams_t.transpose(0,2,3,1)
    model = Model(melgram_input, x)
    prediction = model.predict(melgrams_t)
    ultimate_result = []
    for song_idx, audio_path in enumerate(sorted(glob.glob(audio_path))):
        sorted_result_new = sort_result(tags, prediction[song_idx, :].tolist())
        ultimate_result.append(sorted_result_new)
    mus_index = []
    for i in range(len(ultimate_result[0])):
        mus_index.append(ultimate_result[0][i][0])
    ultimate_df=pd.DataFrame(ultimate_result, columns=mus_index)
    for tag in ultimate_df.keys():
        for num in range(ultimate_df.shape[0]):
            ultimate_df[tag][num] = ultimate_df[tag][num][1]
    reduced_feat = ultimate_df[ultimate_df.columns.values[(np.argsort(importances)[::-1])[:5]]]
    result = clf.predict(reduced_feat)
    for idx, judge in enumerate(result):
        if  judge == 1:
            print("index{} is Chopin!".format(idx))
        else:
            print("index{} is Other!".format(idx))

In [138]:
predict('chopin_for_classification/*.mp3')

index0 is Other!
index1 is Other!
index2 is Other!
index3 is Other!
index4 is Other!
index5 is Other!
index6 is Other!
index7 is Other!
index8 is Other!
index9 is Other!
index10 is Other!
index11 is Other!
index12 is Other!
index13 is Other!
index14 is Other!
index15 is Other!
index16 is Other!
index17 is Other!
index18 is Other!
index19 is Other!
index20 is Other!
index21 is Other!
index22 is Other!
index23 is Other!
index24 is Other!
index25 is Other!
index26 is Other!
index27 is Other!
index28 is Other!
index29 is Other!
index30 is Other!
index31 is Other!
index32 is Other!
index33 is Other!
index34 is Other!
index35 is Other!
index36 is Other!
index37 is Other!
index38 is Other!
index39 is Other!
index40 is Other!
index41 is Other!
index42 is Other!
index43 is Other!
index44 is Other!
index45 is Other!
index46 is Other!
index47 is Other!
index48 is Other!
index49 is Other!
index50 is Other!
index51 is Other!
index52 is Other!
index53 is Other!
index54 is Other!
index55 is Other!
in

In [129]:
predict("Non_Chopin_Music/*.mp3")

index0 is Other!
index1 is Other!
index2 is Other!
index3 is Other!
index4 is Other!
